In [13]:
import re, string, collections
import pandas as pd
import matplotlib.pyplot as plt
import nltk

#from wordcloud import WordCloud
#from nltk import SnowballStemmer
#from nltk.corpus import stopwords
#nltk.download('stopwords')
import pickle
import numpy as np

In [11]:
def cleanData (file, symbols=''):
    tweets = pd.read_csv(file, header=0, names=['ID', 'tweet'], sep='\t')
    data = tweets.get('tweet').tolist()
    clean_data = list()
    for tweet in data:
        url = re.sub(r'http\S+','', tweet)
        simbolos = re.sub(symbols, '', url)
        usuarios = re.sub(r'@\w+', '_user_', simbolos)
        hashtags = re.sub(r'#\w+', '_hashtag_', usuarios)
        clean_data.append(hashtags.lower())
    return data, clean_data

def markData (data):
    frases_pattern = re.compile(
        "(\w+)( )?"
        "[\.|,|;|\?|!]+"
        "( )?(\w+)"
    )
    mark_data = list()
    for tweet in data:
        tweet = ' _B_ ' + tweet + ' _E_ '
        tweet = re.sub(frases_pattern, r'\1 _E_  _B_ \4', tweet, flags=0) #Lo deja para separa ideas
        mark_data.append(tweet)
    return mark_data


def createVocabulary (data_mark, load=False, file='vocabulary_tweets_MX.pk'):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(data_mark)
    if(load):
        with open(file,'rb') as vocabulary_file:
            dic_words = pickle.load(vocabulary_file)
            dic_index = pickle.load(vocabulary_file)
            vocabulario = pickle.load(vocabulary_file)
    else:
        vocabulario = list(set(tokens))
        dic_index = dict(zip(range(len(vocabulario)),vocabulario))  #Índice -> Palabra
        dic_words = dict(zip(vocabulario, range(len(vocabulario)))) #Palabra -> Índice
        with open(file, 'wb') as vocabulary_file:
            pickle.dump(dic_words, vocabulary_file)
            pickle.dump(dic_index, vocabulary_file)
            pickle.dump(vocabulario, vocabulary_file)
    return vocabulario,tokens, dic_index, dic_words


    
def genBigrams(dic_words,tokens,delete):
    corpus_index = [dic_words[token] for token in tokens]
    bigrams = list()
    for i in range(len(corpus_index)-1):
        bigram = (corpus_index[i], corpus_index[i+1])
        if bigram not in delete:
            bigrams.append(bigram)
    return bigrams


In [46]:
puntuacion = r'[\}\{¡"$%&\'()¿:=\+[\]*-]'

data, clean_data = cleanData(r'train_MX.tsv', puntuacion)
mark_data = markData(clean_data)

vocabulario,tokens, dic_index, dic_words = createVocabulary (''.join(mark_data))
frequencies = collections.Counter(tokens)

phrase = (dic_words['_E_'],dic_words['_B_'])
bigrams = genBigrams(dic_words,tokens,[phrase])

with open('Corpus.pk', 'wb') as corpus_file:
    pickle.dump(dic_words, corpus_file)
    pickle.dump(dic_index, corpus_file)
    pickle.dump(vocabulario, corpus_file)
    pickle.dump(bigrams,corpus_file)
    

In [32]:
a = [item[0] for item in filter(lambda x: x[1] == 1, frequencies.items())]
len(a)

3415

In [44]:

for a,b in bigrams[:25]:
    print(a,b)
    print(dic_index[a],dic_index[b],"\n")
phrase

3318 1138
_B_ por 

1138 1911
por fin 

1911 2518
fin terminando 

2518 3478
terminando mis 

3478 2506
mis obras 

2506 72
obras y 

72 4088
y ebbos 

4088 4263
ebbos vamos 

4263 1138
vamos por 

1138 3583
por la 

3583 2647
la última 

2647 72
última y 

72 293
y a 

293 4442
a terminar 

4442 2171
terminar todo 

2171 2926
todo gracias 

2926 293
gracias a 

293 3478
a mis 

3478 690
mis santos 

690 4965
santos _E_ 

3318 1546
_B_ egguns 

1546 72
egguns y 

72 293
y a 

293 3194
a mi 

3194 3452
mi hermano 



(4965, 3318)

In [18]:
print("Texto Original:\n",data[66])
print("Texto Limpio:\n",clean_data[66])
print("Texto Marcado:\n",mark_data[66],"\n")

print("Texto Original:\n",data[71])
print("Texto Limpio:\n",clean_data[71])
print("Texto Marcado:\n",mark_data[71],"\n")

print("Texto Original:\n",data[114])
print("Texto Limpio:\n",clean_data[114])
print("Texto Marcado:\n",mark_data[114],"\n")

Texto Original:
 #BullyingAJos "Mi maestra me dio un beso en la salida, por que hice los palitos derechitos..." Pa!rodia! @JosDice
Texto Limpio:
 _hashtag_ mi maestra me dio un beso en la salida, por que hice los palitos derechitos... pa!rodia! _user_
Texto Marcado:
  _B_ #BullyingAJos "Mi maestra me dio un beso en la salida _E_  _B_ por que hice los palitos derechitos..." Pa _E_  _B_ rodia! @JosDice _E_  

Texto Original:
 ¡Ay como sufren!!! Desde tempranito empezaron a cortarse las venas... Mejor vayan a las posadas, aunque sea pescan a la señora que reza
Texto Limpio:
 ay como sufren!!! desde tempranito empezaron a cortarse las venas... mejor vayan a las posadas, aunque sea pescan a la señora que reza
Texto Marcado:
  _B_ ¡Ay como sufren _E_  _B_ Desde tempranito empezaron a cortarse las venas _E_  _B_ Mejor vayan a las posadas _E_  _B_ aunque sea pescan a la señora que reza _E_  

Texto Original:
 Si definieran su primer día del 2017 con una palabra cuál sería? Respondan citando #E